# AWS S3


In [3]:
import os

import boto3

AWS_REGION = os.getenv("AWS_REGION") or ""
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID") or ""
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY") or ""


s3 = boto3.resource(
    service_name="s3",
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

### List Buckets


In [4]:
for bucket in s3.buckets.all():
    print(bucket.name)

cloud-sdk
sjb-test


### List Objects in Bucket


In [5]:
for obj in s3.Bucket("cloud-sdk").objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='cloud-sdk', key='XETRA.parquet')
s3.ObjectSummary(bucket_name='cloud-sdk', key='data/')
s3.ObjectSummary(bucket_name='cloud-sdk', key='data/XETRA_LIST_OF_SYMBOLS-2.csv')


### Read Object


In [6]:
s3.Bucket("cloud-sdk").Object("data/XETRA_LIST_OF_SYMBOLS-2.csv").get().get("Body").read().decode("utf-8")

'Code,Name,Country,Exchange,Currency,Type,Isin\n00XL,"WisdomTree Copper - EUR Daily Hedged EUR",Germany,XETRA,EUR,ETF,JE00B4PDKD43\n00XN,"WisdomTree Gold - EUR Daily Hedged EUR",Germany,XETRA,EUR,ETF,JE00B4RKQV36\n00XP,"WisdomTree Natural Gas - EUR Daily Hedged EUR",Germany,XETRA,EUR,ETF,JE00B6XF0923\n00XR,"WisdomTree Silver - EUR Daily Hedged EUR",Germany,XETRA,EUR,ETF,JE00B5SV2703\n00XS,"WisdomTree Wheat - EUR Daily Hedged EUR",Germany,XETRA,EUR,ETF,JE00B78NNK09\n00XT,"WisdomTree Brent Crude Oil - EUR Daily Hedged EUR",Germany,XETRA,EUR,ETF,JE00B7305Z55\n04Q,"Nordea Bank Abp",Germany,XETRA,EUR,"Common Stock",FI4000297767\n0B2,"Bawag Group AG",Germany,XETRA,EUR,"Common Stock",AT0000BAWAG2\n0GZA,"RICI Enhanced Natural Gas (ER) Index EUR Hedge ETC",Germany,XETRA,EUR,ETC,DE000PZ9REG5\n0GZB,"RICI Enhanced Copper (ER) Index EUR Hedge ETC",Germany,XETRA,EUR,ETC,DE000PZ9REC4\n0GZC,"RICI Enhanced Nickel (ER) Index EUR Hedge ETC",Germany,XETRA,EUR,ETC,DE000PZ9REN1\n0GZD,"RICI Enhanced Ind Metl

### Read Data using Polars


In [7]:
import polars as pl

storage_options = {
    "aws_access_key_id": AWS_ACCESS_KEY_ID,
    "aws_secret_access_key": AWS_SECRET_ACCESS_KEY,
    "aws_region": AWS_REGION,
}

df = pl.read_parquet("s3://cloud-sdk/XETRA.parquet", storage_options=storage_options)
df

Code,Name,Country,Exchange,Currency,Type,Isin
str,str,str,str,str,str,str
"""00XL""","""WisdomTree Cop…","""Germany""","""XETRA""","""EUR""","""ETF""","""JE00B4PDKD43"""
"""00XN""","""WisdomTree Gol…","""Germany""","""XETRA""","""EUR""","""ETF""","""JE00B4RKQV36"""
"""00XP""","""WisdomTree Nat…","""Germany""","""XETRA""","""EUR""","""ETF""","""JE00B6XF0923"""
"""00XR""","""WisdomTree Sil…","""Germany""","""XETRA""","""EUR""","""ETF""","""JE00B5SV2703"""
"""00XS""","""WisdomTree Whe…","""Germany""","""XETRA""","""EUR""","""ETF""","""JE00B78NNK09"""
"""00XT""","""WisdomTree Bre…","""Germany""","""XETRA""","""EUR""","""ETF""","""JE00B7305Z55"""
"""04Q""","""Nordea Bank Ab…","""Germany""","""XETRA""","""EUR""","""Common Stock""","""FI4000297767"""
"""0B2""","""Bawag Group AG…","""Germany""","""XETRA""","""EUR""","""Common Stock""","""AT0000BAWAG2"""
"""0GZA""","""RICI Enhanced …","""Germany""","""XETRA""","""EUR""","""ETC""","""DE000PZ9REG5"""


In [8]:
import duckdb

duck = duckdb.connect()
duck.execute(f"SET s3_access_key_id='{AWS_ACCESS_KEY_ID}'")
duck.execute(f"SET s3_secret_access_key='{AWS_SECRET_ACCESS_KEY}'")
duck.execute(f"SET s3_region='{AWS_REGION}'")

duck.read_parquet("s3://cloud-sdk/XETRA.parquet")

┌─────────┬──────────────────────────────────────────────┬─────────┬──────────┬──────────┬──────────────┬──────────────┐
│  Code   │                     Name                     │ Country │ Exchange │ Currency │     Type     │     Isin     │
│ varchar │                   varchar                    │ varchar │ varchar  │ varchar  │   varchar    │   varchar    │
├─────────┼──────────────────────────────────────────────┼─────────┼──────────┼──────────┼──────────────┼──────────────┤
│ 00XL    │ WisdomTree Copper - EUR Daily Hedged EUR     │ Germany │ XETRA    │ EUR      │ ETF          │ JE00B4PDKD43 │
│ 00XN    │ WisdomTree Gold - EUR Daily Hedged EUR       │ Germany │ XETRA    │ EUR      │ ETF          │ JE00B4RKQV36 │
│ 00XP    │ WisdomTree Natural Gas - EUR Daily Hedged …  │ Germany │ XETRA    │ EUR      │ ETF          │ JE00B6XF0923 │
│ 00XR    │ WisdomTree Silver - EUR Daily Hedged EUR     │ Germany │ XETRA    │ EUR      │ ETF          │ JE00B5SV2703 │
│ 00XS    │ WisdomTree Wheat - E